In [11]:
import pandas as pd
import torch
import torch.nn as nn

from model.losses import PinballLoss
from pathlib import Path

In [12]:
PATH_DATA = Path.cwd() / 'data' / 'files'

In [13]:
data = pd.read_pickle((PATH_DATA / 'tests.pkl'))

y_pred = data['pred'] 
target = data['target']

quantiles = torch.tensor([0.025, 0.05, 0.1, 0.15, 0.85, 0.9, 0.95, 0.975])

Check quantileloss functions obtained from [here](https://www.yuritan.nl/posts/prediction_uncertainty/)

In [14]:
def LoopLoss(preds, target, quantiles):
    def _tilted_loss(q, e):
        return torch.max((q-1) * e, q * e).unsqueeze(1)

    err = target - preds
    losses = [_tilted_loss(q, err[:, i])  # calculate per quantile
              for i, q in enumerate(quantiles)]

    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

In [15]:
criterion = PinballLoss(quantiles)
criterion(y_pred, target)

tensor(0.0762, grad_fn=<MeanBackward0>)

In [18]:
pinball = PinballLoss(quantiles)
pinball.forward(y_pred, target)

tensor(0.0762, grad_fn=<MeanBackward0>)